In [3]:
import pandas as pd

# Задание 1#
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [1]:
# в задании не указано, что писать в диапазон 4-4.5, соотв. расширил на него 'высокий рейтинг'
def rt(inp):
    return 'низкий рейтинг' if inp <= 2 else 'средний рейтинг' if inp <= 4 else 'высокий рейтинг'

In [12]:
rt_data = pd.read_csv('ratings.csv')
rt_data['class'] = rt_data['rating'].apply(rt)
rt_data.tail(10)

,userId,movieId,rating,timestamp,class
99994,671,5952,5.0,1063502716,высокий рейтинг
99995,671,5989,4.0,1064890625,средний рейтинг
99996,671,5991,4.5,1064245387,высокий рейтинг
99997,671,5995,4.0,1066793014,средний рейтинг
99998,671,6212,2.5,1065149436,средний рейтинг
99999,671,6268,2.5,1065579370,средний рейтинг
100000,671,6269,4.0,1065149201,средний рейтинг
100001,671,6365,4.0,1070940363,средний рейтинг
100002,671,6385,2.5,1070979663,средний рейтинг
100003,671,6565,3.5,1074784724,средний рейтинг


# Задание 2 #
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.


In [19]:
# load
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

kw_data = pd.read_csv('keywords.csv')
print(f'{geo_data}\n\n{kw_data.head()}')

{'Центр': ['москва', 'тула', 'ярославль'], 'Северо-Запад': ['петербург', 'псков', 'мурманск'], 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

         keyword     shows
0             вк  64292779
1  одноклассники  63810309
2          порно  41747114
3           ютуб  39995567
4      вконтакте  21014195


In [50]:
def geo_check(kw):
# исхожу из того, что в запросе может быть указано несколько городов и без пробелов (т.е. не через split)
    lst=[]
    for k, v in geo_data.items():
        for el in v:
             if el in kw.lower():
                lst.append(k)
    lst=','.join(sorted(list(set(lst))))
    return lst if lst else 'undefined'

# geo_check('testСахалинПетербург')

kw_data['region'] = kw_data['keyword'].apply(geo_check)
print(kw_data.head())
kw_data['region'].value_counts()


         keyword     shows     region
0             вк  64292779  undefined
1  одноклассники  63810309  undefined
2          порно  41747114  undefined
3           ютуб  39995567  undefined
4      вконтакте  21014195  undefined


undefined             99253
Центр                   385
Северо-Запад            266
Дальний Восток           94
Северо-Запад,Центр        2
Name: region, dtype: int64

# Задание 3 (бонусное)#
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [97]:
# а можно я чуть оптимизирую задание? судя по файлу, общий паттерн года выпуска - (xxxx)
# плюс за все года. так интереснее. ограничить 1950-2010 не составит труда в production_year
import re
mv_data = pd.read_csv('movies.csv')
rt_data = pd.read_csv('ratings.csv')

def production_year(mv_name):
    pattern = r'(?:\()(\d{4})(?:\))'
    mvy = re.search(pattern, mv_name)
    return int(mvy[1]) if mvy else 1900

mv_data['year'] = mv_data['title'].apply(production_year)
mv_data_l = mv_data.set_index('movieId').T.to_dict('list')

# предполагаю, что с join было бы удобнее
def production_year_rt(mv_id):
    return mv_data_l[mv_id][2] if mv_id in mv_data_l.keys() else 1900

rt_data['year'] = rt_data['movieId'].apply(production_year_rt)
rt_data.groupby('year').mean().sort_values('rating', ascending=False)['rating']

year
1921    4.416667
1902    4.333333
1928    4.261905
1917    4.250000
1918    4.250000
          ...   
2013    3.356973
2000    3.355945
2016    3.217742
1915    3.000000
1919    3.000000
Name: rating, Length: 104, dtype: float64